# Create and Transform 4 Random 4 Quibit States onto Specific Target State Each (Task 2)

### A. Introduction to the Approach Adapted  

#### We approach this task in a very simple 2 step process using Pennylane and PyTorch. 

    1. Construct & utilize a 4 wire circuit to obtain best-parmeters that maximize the similarity between the transformed random state and the target state (i.e the fidelity between the input & target states). 

    2. This can be extended to function as a classifier (& transformer), noting that each set of trained parameters will work best at transforming only the random state(modulo a global phase) that it was trained on to the correct target state. 

Other cuter solutions can include a **softmax function** with the **Forest-SDK**, or a **log-loss** type function to map distributions or any alternatively formulated cost function that obtains best parameters for transforming all 4 random states.

Here keeping I stick with a simple implementation. Thanks for the read and organizing this!

In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.autograd import Variable
import pandas as pd

In [2]:
from IPython.display import display, Latex

### B. Setting up the Circuit Input Paramters

In [3]:
#######################################################################################
#                               Specifying Circuit Parameters                         #
#######################################################################################

#Lets begin by predefining parameters necessary for the code. 
Paulis = Variable(torch.zeros([3, 2, 2], dtype=torch.complex128), requires_grad=False)
Paulis[0] = torch.tensor([[0, 1], [1, 0]])
Paulis[1] = torch.tensor([[0, -1j], [1j, 0]])
Paulis[2] = torch.tensor([[1, 0], [0, -1]])



# We require 4 quibits in our circuit, which are specified.
nr_qubits = 4
# We can use multiple layers of the transforming circuit gates to perform the various transformations. In this case we
# stick with a canonical 2 layers of transforming components.
nr_layers = 2
# We require 4 random 4 quibit states.
n_random_states = 4
#set seed for reproducability 
np.random.seed(41)
# Initialize Random Qubit States. These will be fed as input to the circuit as angles. 
input_state_set = np.random.normal(0, np.pi, (n_random_states,nr_qubits, 3), requires_grad=False)
#Required target states as identified in the problem
target_state_set=[[0,0,1,1],[0,1,0,1],[1,0,1,0.],[1,1,0,0]]

### C. Defining the Circuit Elements

In [4]:
#######################################################################################
#                               Circuit Elements                                      #
#######################################################################################

# Input layer identified by as an SO(3) representation of the SU(2) states.
def input_layer(input_state):
    for i in range(nr_qubits):
        qml.RX(input_state[i, 0], wires=i)
        qml.RX(input_state[i, 1], wires=i)
        qml.RZ(input_state[i, 2], wires=i)

# Layers for the circuit that will perform 
# the necessary state transformations.

def layer(params, j):
    #Our transforming circuit will have multiple layers with 2 sets of rotations and some CNOT gates.
    for i in range(nr_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])

    
# Cost function to be used later to map input vector to target vector.
def cost(params,input_state,target_state_density_matrix):
    # This cost function maximizes the fidelity between the input state and the target state. The circuit code below 
    # makes it obvious how this is made possible. 
    return 1-torch.abs(circuit(params,input_state,target_state_density_matrix))

# Fancy printing for no good reason.
def fancy_print(step,interval):
    if step//interval%4 == 1: 
        return '\U00002196 '
    elif step//interval%4 == 2: 
        return '\U00002197 '
    elif step//interval%4 == 3: 
        return '\U00002198 '
    elif step//interval%4 == 0: 
        return '\U00002199 '
    

device=qml.device('default.qubit', wires=nr_qubits)

#Lazy canonicaliztion of the forms of various vectors.
@qml.qnode(device, interface="torch")
def target_state_vectors(basis_vals):
    # Identifies target states as needed. 
    qml.BasisState(np.array(basis_vals), wires=[0, 1, 2,3])
    return qml.state()


target_state_vector_set=[target_state_vectors(target_state) for target_state in target_state_set]

#We identify operators needed to compute an inner product between the input state and the target vectors.
target_state_density_matrix_set=[torch.outer(target_state.conj(),target_state) for target_state in target_state_vector_set]


# We setup empty dictionaries to store parameters necessary for transforming each of the 4 states. Other 
# solutions instead of the one adopted here could include minimizing the cost across all 4 states simultaneously or a log-loss function.
best_params={}
best_cost={}


# We optimize parameters so as to transform each of the random states to their respective target states. We then
# only store the best parameters obtained for each random state's transformation to the correct target state.
for i in range(4): 
    
    #Identify the input random state and target state corresponding to this iteration. 
    #This way we iterate over all 4 states finding 4 sets of parameters. 
    input_state = input_state_set[i]
    target_state = target_state_vector_set[i]
    target_state_density_matrix=target_state_density_matrix_set[i]
    
    
    #Reset torch variables for each iteration. 
    transformer_params = np.zeros((nr_qubits, nr_layers, 2))
    transformer_params = Variable(torch.tensor(transformer_params), requires_grad=True)
    
    #This is the quantum circuit which is optimized later.
    @qml.qnode(device, interface="torch")
    def circuit(params,input_state=input_state,target_state_density_matrix=target_state_density_matrix):
        
        # Map random state to input state of circuit using function "input_layer" 
        #(essentially via SO(3) angles of the random associated SU(2) state. 
        input_layer(input_state)

        
        #Add circuit elements needed for transformations
        for ith_layer in range(nr_layers):
                layer(params, ith_layer)
                
        #Circuit returns overlap between transformed input state and target state.
        return qml.expval(qml.Hermitian(target_state_density_matrix, wires=[0,1,2,3]))

    #######################################################################################
    # Optimization Routine (runs each time for each state, we only store the parameters)  #                                                         #
    #######################################################################################

    opt = torch.optim.Adam([transformer_params], lr=0.1)

    # We optimize our parameters over 150 iterations.
    steps = 150

    # Initialize empty variables to store best parameters
    best_cost[i] = cost(transformer_params,input_state,target_state_density_matrix)
    best_params[i] = np.zeros((nr_qubits, 3))
 
        
    # Perform optimization steps = 150 times.
    for n in range(steps):
        opt.zero_grad()
        loss = cost(transformer_params,input_state,target_state_density_matrix)
        loss.backward()
        opt.step()
        print(fancy_print(n,5)+"Cost after {} steps is {:.4f}".format(n,cost(transformer_params,input_state,target_state_density_matrix)),end='\r')
        # keeps track of best parameters
        if loss < best_cost[i]:
            best_cost[i] = loss
            best_params[i] = transformer_params
    print('Best parameters for transforming state {} obtained.'.format(i+1)+'\n'+
         '\U00002192 '+'Final fidelity between input & target: {}'.format(1-best_cost[i].data.numpy().item()))

    
#Simple function to see if the transformations on input states correctly produce the required target state.
@qml.qnode(device, interface="torch")
def check_output_state(params,input_state,target_state_density_matrix=None):
    # Initialize starting quibits based on input parameters. 
    for i in range(nr_qubits):
        qml.RX(input_state[i, 0], wires=i)
        qml.RX(input_state[i, 1], wires=i)
        qml.RZ(input_state[i, 2], wires=i)

    #Circuit elements for the transformations.
    for ith_layer in range(nr_layers):
            layer(params, ith_layer)

    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

Best parameters for transforming state 1 obtained.
→ Final fidelity between input & target: 0.9999999337906988
Best parameters for transforming state 2 obtained.
→ Final fidelity between input & target: 0.999999895133897
Best parameters for transforming state 3 obtained.
→ Final fidelity between input & target: 0.9999997893854811
Best parameters for transforming state 4 obtained.
→ Final fidelity between input & target: 0.9999999767861912


In [5]:
# We can verify if we are truly producing the correct output states through our rotations. 
# As we can see below using the sigma_z eigenvalues that the states map as expected.
for i in range(4):
    print(check_output_state(best_params[i],input_state_set[i],target_state_density_matrix_set[i]))

tensor([ 1.0000,  1.0000, -1.0000, -1.0000], dtype=torch.float64,
       grad_fn=<_TorchInterfaceBackward>)
tensor([ 1.0000, -1.0000,  1.0000, -1.0000], dtype=torch.float64,
       grad_fn=<_TorchInterfaceBackward>)
tensor([-1.0000,  1.0000, -1.0000,  1.0000], dtype=torch.float64,
       grad_fn=<_TorchInterfaceBackward>)
tensor([-1.0000, -1.0000,  1.0000,  1.0000], dtype=torch.float64,
       grad_fn=<_TorchInterfaceBackward>)


In [6]:
#######################################################################################
# Using the circuit with 4 sets of identified best parameters as a classifier         #
#######################################################################################

# We use argmax to package all of the parameters obtained into one simple function that will 
# transform a sequence of randomly sampled our created random states into the required target outputs. 

# A cuter solution for more complicated data can use the softmax function for instance.

def format_output(sigmaZ_Eigenvalue): 
    if round(sigmaZ_Eigenvalue,5) == 1: 
        return '0' 
    elif round(sigmaZ_Eigenvalue,5) == -1:
        return '1' 

def simple_classifier_and_transformer(test_state):
    
    # We generate a list of inner products seeing which of the 4 transformations gets a 
    # random state to any one of the expected target states.
    
    possible_transformations=[circuit(best_params[i],test_state,target_state_density_matrix_set[i]).data.numpy() for i in range(4)]
    
    
    # We then pick the index with the largest value using argmax to pick the correct output state for the 
    # corresponding input state, this way in some sense we both classify the input states and transform them. 
    # Note that the classification will only work though if we are acting on our 4 created random states.
    
    transformer_index=np.argmax(possible_transformations)
    output_state_sigmaZ_eigenvalues=check_output_state(best_params[transformer_index],test_state,target_state_density_matrix_set[transformer_index])
    
    output_state_Formatted='|'+''.join([format_output(sigmaZ_Eigenvalue) for sigmaZ_Eigenvalue in output_state_sigmaZ_eigenvalues.data.numpy()])+'\U00003009'
    
    return output_state_Formatted

In [7]:
# Formatted slightly we see that we have correctly transformed our 4 random input states 
# to the required target states.

def format_target_state_toPrint(target_state):
    return '|'+''.join([str(int(state_idx)) for state_idx in target_state])+'\U00003009'

States=[('Random State '+str(i),input_state_set[i]) for i in range(4)] 
df_output_summary=pd.DataFrame(States,columns=['State','Random State Parameters'])
df_output_summary['Desired State']=np.array([format_target_state_toPrint(target_state) for target_state in target_state_set])
df_output_summary['Output State']=df_output_summary.apply(lambda x: simple_classifier_and_transformer(x['Random State Parameters']),axis=1)

df_output_summary

,State,Random State Parameters,Desired State,Output State
0,Random State 0,"[[-0.8504678453845208, 0.3293898718219555, 0.7...",|0011〉,|0011〉
1,Random State 1,"[[-2.421010922477574, -0.10590714557329604, -3...",|0101〉,|0101〉
2,Random State 2,"[[3.770100263642585, -5.725926911263035, 0.846...",|1010〉,|1010〉
3,Random State 3,"[[-1.7854005727964657, -6.633844720434688, -2....",|1100〉,|1100〉


## Classifying Using A Randomly Resampled List of our Previously Created Random Input States

In [8]:
# Let's generate a sequence our random states consisting of a random sampling of a 10 of these states. 
random_resampled_set_of_4states=[]
for i in range(10):
    random_state_to_addtolist=np.random.randint(low=0,high=3+1)
    random_resampled_set_of_4states+=[('Random State '+str(random_state_to_addtolist+1),input_state_set[random_state_to_addtolist])] 

In [9]:
df_check_transformations=pd.DataFrame(random_resampled_set_of_4states,columns=['State','Random State Parameters'])

In [10]:
df_check_transformations['Output State']=df_check_transformations.apply(lambda x: simple_classifier_and_transformer(x['Random State Parameters']),axis=1)

In [11]:
# We see that **without** specifying which state was fed to the circuit we have transformed the 
# input states correctly to the output states, in that sense both classifying and transforming the data,
# this only works since the data here is straight forward :). 

df_check_transformations

,State,Random State Parameters,Output State
0,Random State 2,"[[-2.421010922477574, -0.10590714557329604, -3...",|0101〉
1,Random State 2,"[[-2.421010922477574, -0.10590714557329604, -3...",|0101〉
2,Random State 3,"[[3.770100263642585, -5.725926911263035, 0.846...",|1010〉
3,Random State 3,"[[3.770100263642585, -5.725926911263035, 0.846...",|1010〉
4,Random State 1,"[[-0.8504678453845208, 0.3293898718219555, 0.7...",|0011〉
5,Random State 4,"[[-1.7854005727964657, -6.633844720434688, -2....",|1100〉
6,Random State 2,"[[-2.421010922477574, -0.10590714557329604, -3...",|0101〉
7,Random State 4,"[[-1.7854005727964657, -6.633844720434688, -2....",|1100〉
8,Random State 3,"[[3.770100263642585, -5.725926911263035, 0.846...",|1010〉
9,Random State 3,"[[3.770100263642585, -5.725926911263035, 0.846...",|1010〉


## Conclusion
#### We obtain the following transformations as desiered for the 4 random states.

In [12]:
def format_target_state_toPrint(target_state):
    return '|'+''.join([str(int(state_idx)) for state_idx in target_state])+'\U00003009'

States=[('Random State '+str(i),input_state_set[i]) for i in range(4)] 
df_output_summary=pd.DataFrame(States,columns=['State','Random State Parameters'])
df_output_summary['Desired State']=np.array([format_target_state_toPrint(target_state) for target_state in target_state_set])
df_output_summary['Output State']=df_output_summary.apply(lambda x: simple_classifier_and_transformer(x['Random State Parameters']),axis=1)

df_output_summary

,State,Random State Parameters,Desired State,Output State
0,Random State 0,"[[-0.8504678453845208, 0.3293898718219555, 0.7...",|0011〉,|0011〉
1,Random State 1,"[[-2.421010922477574, -0.10590714557329604, -3...",|0101〉,|0101〉
2,Random State 2,"[[3.770100263642585, -5.725926911263035, 0.846...",|1010〉,|1010〉
3,Random State 3,"[[-1.7854005727964657, -6.633844720434688, -2....",|1100〉,|1100〉


### Feeding this simple classifier/transformer states different from the 4 random states trained on, will, for most such alternate states result in a rotation onto a state which is a superposition of the target states, unless the input state corresponds to a entangled state.